In [1]:
! pip install opencv-python numpy

In [ ]:
import cv2
import numpy as np
import os

# --- Загружаем видео ---
video_path = "video.mp4"  
cap = cv2.VideoCapture(video_path)

# --- Проверяем, открылось ли видео ---
if not cap.isOpened():
    print(f"Ошибка: не удалось открыть видеофайл {video_path}")
    exit()

# --- Читаем первый кадр ---
ret, old_frame = cap.read()
if not ret:
    print("Ошибка: не удалось считать первый кадр.")
    cap.release()
    exit()

old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# --- Определяем ключевые точки методом Shi-Tomasi ---
feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

# --- Параметры Лукаса-Канаде ---
lk_params = dict(winSize=(15, 15), maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# --- Создаем маску для отрисовки траекторий ---
mask = np.zeros_like(old_frame)

# --- Настройки записи видео ---
output_video_path = "output_video.mp4"
frame_width = int(cap.get(3))  # Ширина кадра
frame_height = int(cap.get(4))  # Высота кадра
fps = int(cap.get(cv2.CAP_PROP_FPS))  # FPS исходного видео

# --- Кодек H.264 для `.mp4` ---
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Используем H.264 (mp4v)
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# --- Обработка видео ---
while True:
    ret, frame = cap.read()
    if not ret:
        break  # Выход, если видео закончилось

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # --- Оптический поток методом Лукаса-Канаде ---
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Отбираем успешные точки
    if p1 is not None and st is not None:
        good_new = p1[st == 1]
        good_old = p0[st == 1]

        # --- Отрисовка траекторий ---
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            cv2.line(mask, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)  # Линии движения
            cv2.circle(frame, (int(a), int(b)), 5, (0, 0, 255), -1)  # Текущие точки

        # Обновляем точки
        p0 = good_new.reshape(-1, 1, 2)

    # Объединяем кадр с маской
    output = cv2.add(frame, mask)

    # --- Записываем обработанный кадр в видеофайл ---
    out.write(output)

    # Обновляем предыдущий кадр
    old_gray = frame_gray.copy()

# --- Закрываем файлы ---
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Обработанное видео сохранено как: {output_video_path}")

Обработанное видео сохранено как: output_video.mp4
